In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."


TensorFlow version:  2.16.1


In [3]:
import tensorboard
tensorboard.__version__


'2.16.2'

In [6]:
import os
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import LSTM,GRU,Dense,MaxPooling1D,Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import add,Conv1D
from tensorflow.keras.layers import Input,BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping




from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

In [7]:
model = keras.models.Sequential()
model.add(Conv1D(2048, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(52, 1)))

model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(BatchNormalization())

model.add(Conv1D(1024, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(512, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
model.add(BatchNormalization())

model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

# Compile the model
optimiser = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=optimiser, loss='mean_squared_error', metrics=['mae'])

C:\Users\u144572\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:

import pandas as pd
import re
import os

# Lista plików (zakładając, że masz już listę plików, np. z os.listdir)
file_list = os.listdir('own_data')  # Zastąp 'ścieżka_do_twojego_folderu' odpowiednią ścieżką

# Inicjalizacja listy do przechowywania danych
data = []

# Iteracja po liście plików
for filename in file_list:
    # Wydobywanie liczby całkowitej z nazwy pliku przy użyciu wyrażenia regularnego
    match = re.search(r'\d+', filename)  # Zakładając, że liczba całkowita składa się z co najmniej jednej cyfry
    if match:
        integer = int(match.group())
        data.append({'cycles': integer, 'filename': filename[:-4]})

# Konwersja listy do DataFrame
df = pd.DataFrame(data, columns=['cycles', 'filename'])
# print(type(df["cycles"][0]))
df

import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split

# Parametry zduplikowania
duplication_factor = 1  # Ilość zduplikowania danych

# Wczytanie danych z CSV
audio_data = "own_data/"
# df = pd.read_csv('cycles_count_own_data.csv', names=['Start', 'filename'], header=0)
print(df)

def mfcc_feature_extraction_rr(dir_):
    X_ = []
    y_ = []
    data = df
    features = 50
    for soundDir in os.listdir(dir_):
        if soundDir[-3:] == 'wav':               
            data_x, sampling_rate = librosa.load(dir_ + soundDir, res_type='kaiser_fast')
            mfccs = np.mean(librosa.feature.mfcc(y=data_x, sr=sampling_rate, n_mfcc=features).T, axis=0)
            # Dodajemy cechy i etykiety
            X_.append(mfccs)
            y_.append(list(data[data['filename'] == (soundDir[:-4])]['cycles'])[0])
    
    # Zduplikowanie danych
    X_data = np.tile(np.array(X_), (duplication_factor, 1))
    y_data = np.tile(np.array(y_), duplication_factor)
    
    return X_data, y_data

# Wywołanie funkcji ekstrakcji cech
res_data, res_y = mfcc_feature_extraction_rr(audio_data)


# # Podział danych na zbiory treningowe, walidacyjne i testowe
x_train, x_val, y_train, y_val = train_test_split(res_data, res_y, test_size=0.2, random_state=10)

# # Wyświetlenie rozmiarów zbiorów danych
# print(x_train.shape, x_val.shape)
# print(y_train.shape, y_val.shape)

x_train_lstm = np.expand_dims(x_train,axis=2)
x_val_lstm = np.expand_dims(x_val,axis=2)

# # Podział danych na zbiory treningowe, walidacyjne i testowe
# x_train, x_val, y_train, y_val = train_test_split(res_data, res_y, test_size=0.175, random_state=10)
# x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.175, random_state=10)

# # Wyświetlenie rozmiarów zbiorów danych
# print(x_train.shape, x_val.shape, x_test.shape)
# print(y_train.shape, y_val.shape, y_test.shape)

# x_train_lstm = np.expand_dims(x_train,axis=2)
# x_val_lstm = np.expand_dims(x_val,axis=2)
# x_test_lstm = np.expand_dims(x_test,axis=2)

from tensorflow.keras.models import load_model
import tensorflow as tf

# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

history = model.fit(
    x_train_lstm, y_train, batch_size=16, epochs=2, validation_data=(x_val_lstm, y_val),
  )


    cycles       filename
0       10          10(1)
1       10          10(2)
2       10          10(3)
3       10             10
4       11   11 normalnie
..     ...            ...
59       9  ustami_idąc_9
60       6            Ła6
61       6         Łagie6
62       4          €-€-4
63       7           €€_7

[64 rows x 2 columns]
Epoch 1/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - loss: 76.2626 - mae: 8.0034 - val_loss: 64.9277 - val_mae: 7.7845
Epoch 2/2
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 439ms/step - loss: 63.0686 - mae: 7.3536 - val_loss: 64.0902 - val_mae: 7.7297


In [12]:
%tensorboard --logdir logs


Reusing TensorBoard on port 6006 (pid 25152), started 0:02:49 ago. (Use '!kill 25152' to kill it.)